<a href="https://colab.research.google.com/github/vishvjitsinh24/Q-A_Evaluation/blob/main/ollama_Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install lshw
!sudo lshw -C display

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
lshw is already the newest version (02.19.git.2021.06.19.996aaad9c7-2build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
  *-display
       description: 3D controller
       product: TU104GL [Tesla T4]
       vendor: NVIDIA Corporation
       physical id: 4
       bus info: pci@0000:00:04.0
       version: a1
       width: 64 bits
       clock: 33MHz
       capabilities: msix pm bus_master cap_list
       configuration: driver=nvidia latency=0
       resources: iomemory:30-2f iomemory:30-2f irq:11 memory:c0000000-c0ffffff memory:380000000-38fffffff memory:390000000-391ffffff


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!ollama serve

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Error: listen tcp 127.0.0.1:11434: bind: address already in use


In [ ]:
# !pip install colab-xterm
# %load_ext colabxterm
# %xterm

In [ ]:
!ollama

Usage:
  ollama [flags]
  ollama [command]

Available Commands:
  serve       Start ollama
  create      Create a model
  show        Show information for a model
  run         Run a model
  stop        Stop a running model
  pull        Pull a model from a registry
  push        Push a model to a registry
  list        List models
  ps          List running models
  cp          Copy a model
  rm          Remove a model
  help        Help about any command

Flags:
  -h, --help      help for ollama
  -v, --version   Show version information

Use "ollama [command] --help" for more information about a command.


In [ ]:
!pip install -q Ollama

In [ ]:
!ollama pull llama3

In [ ]:
import ollama
import json

def check_relevance_with_llm(question, context, answer):
    """
    Checks the relevance and correctness of an answer using a local LLM.

    Args:
        question (str): The original question.
        context (str): The source text or information used to generate the answer.
        answer (str): The answer to be evaluated.

    Returns:
        dict: The LLM's evaluation and a score.
    """
    prompt = f"""
    You are an expert Q&A evaluator. Your task is to check if a given answer is
    relevant, accurate, and correct based on a provided question and context.

    Question: {question}
    Context: {context}
    Answer: {answer}

    Please provide a concise and objective evaluation in a structured JSON format.
    Your evaluation should include:
    - 'is_relevant': A boolean (true/false) indicating if the answer directly addresses the question.
    - 'is_correct': A boolean (true/false) indicating if the answer's facts are supported by the context.
    - 'score': A numerical score from 0 to 10, where 10 is a perfect response.
    - 'reasoning': A brief explanation for the score.

    The output must be a single JSON object.
    """

    try:
        response = ollama.chat(
            model='llama3',
            format='json',
            messages=[{'role': 'user', 'content': prompt}]
        )
        # Assuming the LLM returns a JSON string, we need to parse it.
        # This can sometimes be tricky, so error handling is key.
        evaluation_json = json.loads(response['message']['content'])
        return evaluation_json

    except ollama.ResponseError as e:
        return {"error": str(e), "message": "Ollama service or model is not available."}
    except json.JSONDecodeError:
        return {"error": "JSON parsing failed", "message": "LLM did not return a valid JSON format."}


# --- Example Usage ---

# Case 1.1: A good, relevant, and correct answer
question = "First step for minor burn?"
context = "Cool the burn under cool running water for 10–15 minutes. Do not use ice."
answer = "Cool under running water for 10–15 minutes."
result1 = check_relevance_with_llm(question, context, answer)
print(f"Question: '{question}'\nContext: '{context}'\nAnswer: '{answer}'")
print("LLM Evaluation:", result1)
print("-" * 50)

# Case 1.2: An irrelevant and incorrect answer
question = "First step for minor burn?"
context = "Cool the burn under cool running water for 10–15 minutes. Do not use ice."
answer = "Apply butter to the burn."
result2 = check_relevance_with_llm(question, context, answer)
print(f"Question: '{question}'\nContext: '{context}'\nAnswer: '{answer}'")
print("LLM Evaluation:", result2)

# Case 2.1: A good, relevant, and correct answer
question = "Legal drinking age in Ontario?"
context = "In Ontario, alcohol may only be served to people aged 19 or older."
answer = "19 years old."
result1 = check_relevance_with_llm(question, context, answer)
print(f"Question: '{question}'\nContext: '{context}'\nAnswer: '{answer}'")
print("LLM Evaluation:", result1)
print("-" * 50)

# Case 2.2: An irrelevant and incorrect answer
question = "Legal drinking age in Ontario?"
context = "In Ontario, alcohol may only be served to people aged 19 or older."
answer = "21 years old."
result2 = check_relevance_with_llm(question, context, answer)
print(f"Question: '{question}'\nContext: '{context}'\nAnswer: '{answer}'")
print("LLM Evaluation:", result2)

Question: 'First step for minor burn?'
Context: 'Cool the burn under cool running water for 10–15 minutes. Do not use ice.'
Answer: 'Cool under running water for 10–15 minutes.'
LLM Evaluation: {'is_relevant': True, 'is_correct': True, 'score': 10, 'reasoning': 'The answer directly addresses the question, and its facts are supported by the context. The context provides guidance on how to cool a minor burn, which is exactly what the answer describes. The accuracy of the information is also confirmed by the context.'}
--------------------------------------------------
Question: 'First step for minor burn?'
Context: 'Cool the burn under cool running water for 10–15 minutes. Do not use ice.'
Answer: 'Apply butter to the burn.'
LLM Evaluation: {'evaluation': {'is_relevant': False, 'is_correct': False, 'score': 2, 'reasoning': 'The answer does not address the first step for minor burns, which is to cool it under cool running water. Additionally, applying butter to a burn is not supported by 